# Setup

In [1]:
import sys
import os
import numpy as np
# sys.path.append(os.path.abspath('/content/gdrive/MyDrive'))
sys.path.insert(0, 'evoman')

local_dir = os.path.dirname('evoman') 

In [2]:
from environment import Environment

pygame 2.0.1 (SDL 2.0.14, Python 3.7.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Environment

In [3]:
# initializes simulation in individual evolution mode, for single static enemy.
os.environ["SDL_VIDEODRIVER"] = "dummy"

experiment_name = 'individual_demo'
env = Environment(experiment_name=experiment_name,
                  enemies=[2],
                  playermode="ai",
                  enemymode="static",
                  level=2,
                  speed="fastest")




MESSAGE: Pygame initialized for simulation.


# NEAT


In [4]:
import neat, multiprocessing, os, pickle
from evoman.controller import Controller

In [5]:
# def g(x):
#   return np.power(2, -(np.square(x / 150)))

# def transform_distance(x):
#   if x == 0:
#     return 0
#   elif x > 0:
#     return g(x)
#   else:
#     return -1 * g(x)

In [6]:
class rnn_controller(Controller):
  def __init__(self, ctrnn, time_const):
    self.ctrnn = ctrnn
    self.time_const = time_const


def control(self, inputs, controller):
    # transform bullet distances
    for i in range(4, 20):
        inputs[i] = transform_distance(inputs[i])

    output = ctrnn.advance(inputs, self.time_const, self.time_const)
    left, right, jump, shoot, release = 0,0,0,0,0

    if output[0] > 0.5:
        left = 1
    else:
        left = 0

    if output[1] > 0.5:
        right = 1
    else:
        right = 0

    if output[2] > 0.5:
        jump = 1
    else:
        jump = 0

    if output[3] > 0.5:
        shoot = 1
    else:
        shoot = 0

    if output[4] > 0.5:
        release = 1
    else:
        release = 0

    return [left, right, jump, shoot, release]
    


In [7]:
TIME_CONST = 0.001
runs_per_net = 1

def eval_genome(genome, config):
  net = neat.ctrnn.CTRNN.create(genome, config, TIME_CONST)
  controller = rnn_controller(net, TIME_CONST)

  fitnesses = []
  for runs in range(runs_per_net):
    net.reset()
    fitness = 0
    f,p,e,t = env.play(pcont=controller)

    fitnesses.append(f)


  return min(fitnesses)

In [ ]:
# Ikrame and Jaspreet: uncomment below
# n = 100
# config_path = os.path.join(local_dir, 'ikrame_jaspreet.cfg')

# Zach and Niklas: uncomment below
n = 10
config_path = os.path.join(local_dir, 'zach_niklas.cfg')

# config_path = os.path.join(local_dir, 'default.cfg')

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, 
                     neat.DefaultSpeciesSet, neat.DefaultStagnation, 
                     config_path)


print("STARTING")
means = []
stddevs = []
for _ in range(5):
  pop = neat.Population(config)
  stats = neat.StatisticsReporter()
  pop.add_reporter(stats)
  pop.add_reporter(neat.StdOutReporter(True))
  pe = neat.ParallelEvaluator(multiprocessing.cpu_count(), eval_genome)
  winner = pop.run(pe.evaluate, n=n)
  means.append(stats.get_fitness_mean())

means_np = np.array(means)
print(f"Overall mean: {means_np.mean()}")
print(f"Overall std: {means_np.std()}")



STARTING

 ****** Running generation 0 ****** 


RUN: run status: enemy: 2; fitness: 39.20698639161586; player life: 0; enemy life: 50.0; time: 328

RUN: run status: enemy: 2; fitness: 39.20698639161586; player life: 0; enemy life: 50.0; time: 328

RUN: run status: enemy: 2; fitness: 39.20698639161586; player life: 0; enemy life: 50.0; time: 328

RUN: run status: enemy: 2; fitness: 39.20698639161586; player life: 0; enemy life: 50.0; time: 328

RUN: run status: enemy: 2; fitness: 30.182888840036796; player life: 0; enemy life: 60.0; time: 336

RUN: run status: enemy: 2; fitness: 30.182888840036796; player life: 0; enemy life: 60.0; time: 336
